## Final Project Submission

Please fill out:
* Student name: Ioana Preoteasa & Fadi Sarraf
* Student pace: full time
* Scheduled project review date/time: THU 12 Dec, 15.15
* Instructor name: Daniel Sanz
* Blog post URL: n/a


# What makes a successful song?
### Spotify Billboard Prediction and Analysis